In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Carga de los datos

df_test = pd.read_csv("./new_classification_test.csv", sep = ",")
df_train = pd.read_csv("./new_classification_train.csv", sep = ",")

In [ ]:
# Mostramos dataset de test
df_test

In [ ]:
# Mostramos dataset de train
df_train

Como podemos ver, los datos de train tienen una columna más respecto a los datos de test. Esta columna es la de la variable a predecir (category)

In [ ]:
df_train.describe()

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df_train)

profile

Como podemos ver, el dataset está formado de 134.571 observaciones. 
En total, hay 37.743 celdas sin datos, por lo que vamos a tener que tener esto en cuenta.

La primera columna es un id, por lo que la podemos borrar, ya que no aporta nada de información, debido a que es un índice.

La segunda columna, es el headline. Hay 133.726 datos distintos y 3 faltantes, por lo que podemos concluir que hay algunos headlines repetidos en el dataset.

Algo similar ocurre con la columna authors. Solo el 20% de los datos son distintos, por lo que hay autores que se repiten. Esto puede ser útil porque tal vez un autor tenga preferencia a escribir noticias de una categoría en particular. Además, un dato no menor es que faltan 24.477 datos de autores. 

En cuanto a la categoria short_description, podemos observar que en su gran mayoría son distintos, pero hay algunas descripciones que se repiten. Además, también hay un gran número de valores faltantes (13.263).

De la columna date podemos decir que hay muchos datos repetidos, ya que solo e 1.7% son valores distintos. Además, no falta ningún valor.

Por último, la columna category, es la de la variable a predecir, Hay 41 categorías distintas en todo el set de datos. 



In [ ]:
# Llenamos datos faltantes:
df_train['headline'].fillna('Unknown headline', inplace=True)
df_test['headline'].fillna('Unknown headline', inplace=True)

In [ ]:
df_train['short_description'].fillna('Unknown short description', inplace=True)
df_test['short_description'].fillna('Unknown short description', inplace=True)

In [ ]:
df_train['authors'].fillna('Unknown authors', inplace=True)
df_test['authors'].fillna('Unknown authors', inplace=True)

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df_train)

profile

In [ ]:
df_train

In [ ]:
# Eliminamos columna de id
# Quitamos la columna id del dataframe porque vimos que era un id autoincremental, el cual era lo mismo que el índice de la fila, por lo tanto, no aportaba información.
df_train.drop(columns=['id'], axis=1, inplace=True)
df_test.drop(columns=['id'], axis=1, inplace=True)

In [ ]:
df_train

In [ ]:
# Cambiamos el tipo de los datos a string, ya que antes eran de tipo object, pero en realidad, todos deberían ser strings
df_train[['headline', 'authors', 'short_description', 'category', 'date']] = df_train[['headline', 'authors', 'short_description', 'category', 'date']].astype('string')
df_test[['headline', 'authors', 'short_description', 'date']] = df_test[['headline', 'authors', 'short_description', 'date']].astype('string')

In [ ]:
df_train.info()

In [ ]:
# Hay 2309 fechas distintas en el dataset
df_train['date'].unique()

In [ ]:
# Hay 41 categorias distintas en el dataset
df_train['category'].unique()

In [ ]:
plt.figure(figsize=(80,40))
df_train['category'].hist()

In [ ]:
#Countplot por categoria
plt.figure(figsize=(50,5))
sns.countplot(data=df_train, x='category')

Como podemos observar, la mayoría de noticias son de la categoría política

El histograma nos da una noción global de como es la distribución de "category". Los valores más frecuentes son:

1 - Politics (21935)

2 - Wellness (11944)

3 - Entertainment (10759)

## VECTORIZACIÓN

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

### Vectorización de headlines

In [ ]:
headlines_corpus = list(df_train['headline'])
headlines_corpus_test = list(df_test['headline'])

headline_vectorizer = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True, max_features=1000, max_df=0.80).fit(headlines_corpus)
headline_vectorizer_test = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True, max_features=1000, max_df=0.80).fit(headlines_corpus_test)

In [ ]:
headline_vectorizer.get_feature_names()

In [ ]:
len(headline_vectorizer.get_feature_names())

In [ ]:
X_train_headlines = headline_vectorizer.transform(headlines_corpus)
X_test_headlines = headline_vectorizer_test.transform(headlines_corpus_test)
print(X_train_headlines.toarray())

### Vectorización de short_description

In [ ]:
short_description_corpus = list(df_train['short_description'])
short_description_corpus_test = list(df_test['short_description'])

shortDescription_vectorizer = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True,max_features=1000, max_df=0.80, ).fit(short_description_corpus)
shortDescription_vectorizer_test = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True,max_features=1000, max_df=0.80, ).fit(short_description_corpus_test)

In [ ]:
shortDescription_vectorizer.get_feature_names()

In [ ]:
len(shortDescription_vectorizer.get_feature_names())

In [ ]:
X_train_short_description = shortDescription_vectorizer.transform(short_description_corpus)
X_test_short_description = shortDescription_vectorizer_test.transform(short_description_corpus_test)
print(X_train_short_description.toarray())

### Vectorización de authors

In [ ]:
authors_corpus = list(df_train['authors'])
authors_corpus_test = list(df_test['authors'])

authors_vectorizer = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True, max_features=1000, max_df=0.80).fit(authors_corpus)
authors_vectorizer_test = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True, max_features=1000, max_df=0.80).fit(authors_corpus_test)

In [ ]:
authors_vectorizer.get_feature_names()

In [ ]:
len(authors_vectorizer.get_feature_names())

In [ ]:
X_train_authors = authors_vectorizer.transform(authors_corpus)
X_test_authors = authors_vectorizer_test.transform(authors_corpus_test)
print(X_train_authors.toarray())

### Ahora concatenamos los vectores de headlines, short_description y authors

In [ ]:
# TRAIN
headline_feature_names = ['headline' + elem for elem in headline_vectorizer.get_feature_names()]
shortDescription_feature_names = ['shortDescription' + elem for elem in shortDescription_vectorizer.get_feature_names()]
authors_feature_names = ['authors' + elem for elem in authors_vectorizer.get_feature_names()]
dfHeadline = pd.DataFrame.sparse.from_spmatrix(X_train_headlines, columns=headline_feature_names)
dfShortDescription = pd.DataFrame.sparse.from_spmatrix(X_train_short_description, columns=shortDescription_feature_names)
dfAuthors = pd.DataFrame.sparse.from_spmatrix(X_train_authors, columns=authors_feature_names)

In [ ]:
# TEST
headline_feature_names_test = ['headline' + elem for elem in headline_vectorizer_test.get_feature_names()]
shortDescription_feature_names_test = ['shortDescription' + elem for elem in shortDescription_vectorizer_test.get_feature_names()]
authors_feature_names_test = ['authors' + elem for elem in authors_vectorizer_test.get_feature_names()]
dfHeadline_test = pd.DataFrame.sparse.from_spmatrix(X_test_headlines, columns=headline_feature_names_test)
dfShortDescription_test = pd.DataFrame.sparse.from_spmatrix(X_test_short_description, columns=shortDescription_feature_names_test)
dfAuthors_test = pd.DataFrame.sparse.from_spmatrix(X_test_authors, columns=authors_feature_names_test)

In [ ]:
dfHeadline

In [ ]:
dfShortDescription

In [ ]:
dfAuthors

In [ ]:
df_train = pd.concat([df_train, dfHeadline, dfShortDescription, dfAuthors], axis=1)
df_test = pd.concat([df_test, dfHeadline_test, dfShortDescription_test, dfAuthors_test], axis=1)

In [ ]:
df_train

In [ ]:
# Eliminamos columnas de headline, short_description y authors
df_train.drop(columns=['headline', 'short_description', 'authors'], inplace=True)
df_test.drop(columns=['headline', 'short_description', 'authors'], inplace=True)

In [ ]:
df_train

## Label Encoders

In [ ]:
from sklearn.preprocessing import LabelEncoder

### Label encoder para "date"


In [ ]:
dateLE = LabelEncoder()

In [ ]:
df_train['date'] = dateLE.fit_transform(df_train['date'])
df_test['date'] = dateLE.fit_transform(df_test['date'])

In [ ]:
df_train

### One-hot encoding para "category"

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
categoryOHE = OneHotEncoder(handle_unknown='ignore')

In [ ]:
df_train['category'].unique()

In [ ]:
categories = df_train['category'].to_numpy()
categories = categories.reshape(-1,1)

In [ ]:
y = categoryOHE.fit_transform(categories).toarray()
print(y)

In [ ]:
categoryOHE.categories_

In [ ]:
X = df_train.drop(['category'], axis=1)
X

## ARMADO DE ÁRBOL DE DECISIÓN

### De profundidad máxima 3, criterio Gini

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
arbol = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)

In [ ]:
# Dividimos train en train y dev
from sklearn.model_selection import train_test_split

X_train_tree, X_dev_tree, Y_train_tree, Y_devtree = train_test_split(
    df_train.drop(['category'], axis=1),
    y,
    test_size=(0.1), random_state=42)

In [ ]:
arbol.fit(X_train_tree, Y_train_tree)

In [ ]:
Y_pred = arbol.predict(X_dev_tree)
Y_pred

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy Arbol:", accuracy_score(Y_pred, Y_devtree))

In [ ]:
# set to 1 the highest element of an array and the rest to 0
def set_to_one(array):
 newArr = np.zeros(array.shape)
 newArr[array.argmax()] = 1
 return newArr

In [ ]:
arr = np.array([0.1,0.2,0.3,0.8,0.5])
set_to_one(arr)

In [ ]:
df_train['category'].unique()

In [ ]:
# iterate over the matrix and set to 1 the highest element of each row
def set_to_one_row(matrix):
	for i in range(len(matrix)):
		row = matrix[i]
		matrix[i] = set_to_one(row)
	return matrix

In [ ]:
def predict_then_save(model, X_test, filename):
    Y_pred = model.predict(X_test)
    print(Y_pred[0])
    Y_pred = set_to_one_row(Y_pred)
    Y_pred_decoded = categoryOHE.inverse_transform(Y_pred).astype(str).reshape(-1)
    with open(filename, 'w') as f:
        f.write('id,category\n')
        for i, pred in enumerate(Y_pred_decoded):
            f.write(str(i) + ',' + str(pred) + '\n')

In [ ]:
predict_then_save(arbol, df_test, 'submission_arbol.csv')

## ARMADO DE RED NEURONAL

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def make_NN_model(output_shapes, input_shape, use_bias, activations, dropouts, optimizer, loss, metric):
    model = Sequential()
    layers = len(output_shapes)
    for i in range(0, layers):
        if i == 0:
            model.add(Dense(output_shapes[i], input_shape=(input_shape,), use_bias=use_bias))
        else: 
            model.add(Dense(output_shapes[i], use_bias=use_bias))
            model.add(Activation(activations[i]))
        if dropouts[i] != None:
            model.add(dropouts[i])
    model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
    return model

### Primera red neuronal

In [ ]:
model = make_NN_model(
  output_shapes=[25, 35, Y_train_tree.shape[1]],
  input_shape=len(df_train.drop(['category'], axis=1).columns),
  use_bias=True,
  activations=[LeakyReLU(alpha=0.3), LeakyReLU(alpha=0.3), 'softmax'],
  dropouts=[Dropout(0.3), None, None],
  optimizer='Adam',
  loss='categorical_crossentropy',
  metric='accuracy' 
)

In [ ]:
early_stopping = EarlyStopping(monitor='accuracy', patience=5)
model.fit(X_train_tree, Y_train_tree, epochs=2000, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
Y_pred_model = model.predict(X_dev_tree)
Y_pred_model

In [ ]:
Y_pred_model = set_to_one_row(Y_pred_model)

In [ ]:
Y_pred_model

In [ ]:
print("Accuracy Arbol:", accuracy_score(Y_pred_model, Y_devtree))

### Segunda red neuronal

In [217]:
# Volvemos a cargar los df para trabajar en copias limpias

df_test = pd.read_csv("./new_classification_test.csv", sep = ",")
df_train = pd.read_csv("./new_classification_train.csv", sep = ",")

In [195]:
from string import punctuation
from collections import Counter
from functools import reduce

def count_punctuation(text):
	counter = Counter(char for char in text if char in punctuation)
	counter_list = counter.values()
	return reduce(lambda x, y: x + y, counter_list, 0)

In [196]:
count_punctuation('Hello, my name is John.')

2

In [178]:
X_train, X_dev, Y_train, Y_dev = train_test_split(
    df_train.drop(['category'], axis=1),
    df_train['category'],
    test_size=(0.1), random_state=42)
X_train

,id,headline,authors,short_description,date
10912,10912,"About 3% Of Americans Sick With The Flu, Surve...",NaN,Authors of the report noted that the findings ...,2014-01-10
59139,59139,Recipe Of The Day: Croissant Pudding,NaN,And croissants play a major role.,2012-11-16
68553,68553,So You're Newly Engaged... Now What?,NaN,"Enjoy the Ride The planning is exciting, but c...",2014-01-12
108983,108983,Even Nonsmoking Teens Are Constantly Exposed T...,"Lindsey Tanner, Associated Press",There's no safe level of secondhand smoke expo...,2016-01-11
3912,3912,Jeb and Hillary Baggage Wars,"Phil Perrier, ContributorFreelance Writer and ...",The conventional wisdom at the moment is that ...,2015-02-23
...,...,...,...,...,...
110268,110268,Wedding Trends 2012: Industry Insiders Choose ...,Stephanie Hallett,What a year it's been for wedding enthusiasts....,2012-12-29
119879,119879,Donald Trump Now Says Humans Somehow Contribut...,Laura Barrón-López,"""There is some connectivity.""",2016-11-22
103694,103694,"Rene Gonzalez, Architect, Designs Stunning Bui...",Brie Dyas,"Here, every room has a great view.",2013-07-12
131932,131932,10 Steps to 'Living in the Moment',"Rea Nolan Martin, Contributor\nAward-winning a...","In a nutshell, to live in the moment means to ...",2014-03-22


In [157]:
df_train

,id,headline,authors,short_description,date,category
0,0,Does Romans 13 Give The President The Right To...,"David J. Dunn, PhD, ContributorTheological tro...",Let any who doubt the lamentable state of theo...,2017-08-12,RELIGION
1,1,How To Throw A Marijuana-Friendly Wedding,NaN,Designate a pot-smoking area. While brides and...,2014-04-09,WEDDINGS
2,2,The New Yorker Finally Realized Us Plebeians D...,Katla McGlynn,NaN,2014-10-23,COMEDY
3,3,The 10 Best U.S. Cities For Retirement,"PureWow, ContributorWhat your girl crush reads...",Next question: When is too early to start coun...,2017-07-10,BUSINESS
4,4,"Justin Bieber Punches Man In The Face, Leaves ...",Carly Ledbetter,This fan got WAY too close for comfort.,2016-11-23,ENTERTAINMENT
...,...,...,...,...,...,...
134566,134566,Added Sugars Make Up 13 Percent Of U.S. Adults...,NaN,A recent study of added sugar consumption in k...,2013-05-01,WELLNESS
134567,134567,Must-Try Bohemian Trends,"W Magazine, Contributor\nW Magazine",Whether it was the suede dresses at Calvin Kle...,2013-11-27,STYLE & BEAUTY
134568,134568,From A Dad Who Supports Breastfeeding: My Boys...,"John Willey, Contributor\nBlogger, DaddysInCha...",What is this world coming to that people would...,2012-02-02,PARENTING
134569,134569,"Bikinis, Texas: Businessman Buys Bankersmith, ...",NaN,"The town was first established in 1913, when t...",2012-07-17,TRAVEL


In [198]:
def add_extra_features(df):
  df['document_length'] = df['short_description'].map(lambda x: len(x))
  df['author_count'] = df['authors'].map(lambda x: len(x.split(',')))
  df['punctuation_count'] = df['short_description'].map(lambda x: count_punctuation(x))
  return df 

In [180]:
def label_encode_date(df):
  dateLE = LabelEncoder()
  df['date'] = dateLE.fit_transform(df['date'])
  return df

In [227]:
def perform_vectorization(df):
  df = label_encode_date(df)
  
  headlines_corpus = list(df['headline'])
  short_description_corpus = list(df['short_description'])
  authors_corpus = list(df['authors'])
  
  headline_vectorizer = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True, max_features=1000, max_df=0.80).fit(headlines_corpus)
  short_description_vectorizer = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True, max_features=1000, max_df=0.80).fit(short_description_corpus)
  authors_vectorizer = CountVectorizer(stop_words='english', dtype=np.int32, lowercase=True, max_features=1000, max_df=0.80).fit(authors_corpus)
  
  _headlines = headline_vectorizer.transform(headlines_corpus)
  _short_description = short_description_vectorizer.transform(short_description_corpus)
  _authors = authors_vectorizer.transform(authors_corpus)
  
  headline_feature_names = ['headline' + elem for elem in headline_vectorizer.get_feature_names()]
  shortDescription_feature_names = ['shortDescription' + elem for elem in shortDescription_vectorizer.get_feature_names()]
  authors_feature_names = ['authors' + elem for elem in authors_vectorizer.get_feature_names()]
  
  dfHeadline = pd.DataFrame.sparse.from_spmatrix(_headlines, columns=headline_feature_names)
  dfShortDescription = pd.DataFrame.sparse.from_spmatrix(_short_description, columns=shortDescription_feature_names)
  dfAuthors = pd.DataFrame.sparse.from_spmatrix(_authors, columns=authors_feature_names)

  print(dfShortDescription)

  df = pd.concat([df, dfHeadline, dfShortDescription, dfAuthors], axis=1)

  df.drop(['headline', 'short_description', 'authors'], axis=1, inplace=True)
  return df

In [181]:
def one_hot_encode(y, encoder):
  categories = y.to_numpy()
  categories = categories.reshape(-1,1)
  return categoryOHE.fit_transform(categories).toarray()

In [206]:
train_categoryOHE = OneHotEncoder(handle_unknown='ignore')
dev_categoryOHE = OneHotEncoder(handle_unknown='ignore')

def do_attribute_engineering_with_split(df):
	df['headline'].fillna('Unknown headline', inplace=True)
	df['short_description'].fillna('Unknown short description', inplace=True)
	df['authors'].fillna('Unknown author', inplace=True)
	df[['headline', 'authors', 'short_description', 'date', 'category']] = df[['headline', 'authors', 'short_description', 'date', 'category']].astype('string')
	df.drop(['id'], axis=1, inplace=True)
	df = add_extra_features(df)

	X_train, X_dev, Y_train, Y_dev = train_test_split(
    df.drop(['category'], axis=1),
    df['category'],
    test_size=(0.1), random_state=42)
  
	return perform_vectorization(X_train), perform_vectorization(X_dev), one_hot_encode(Y_train, train_categoryOHE), one_hot_encode(Y_dev, dev_categoryOHE)

In [183]:
def do_attribute_engineering_without_split(df):
	df['headline'].fillna('Unknown headline', inplace=True)
	df['short_description'].fillna('Unknown short description', inplace=True)
	df['authors'].fillna('Unknown author', inplace=True)
	df[['headline', 'authors', 'short_description', 'date']] = df[['headline', 'authors', 'short_description', 'date']].astype('string')
	df = add_extra_features(df)
  
	return perform_vectorization(df)

In [189]:
count_punctuation(df_train['short_description'][0])

1

In [197]:
df_train['short_description'].map(lambda x: count_punctuation(x))

0         1
1         3
2         0
3         2
4         1
         ..
134566    1
134567    4
134568    5
134569    2
134570    2
Name: short_description, Length: 134571, dtype: int64

In [228]:
df_train = pd.read_csv("./new_classification_train.csv", sep = ",")
X_train_nn_2, X_dev_nn_2, Y_train_nn_2, Y_dev_nn_2 = do_attribute_engineering_with_split(df_train)

        shortDescription000  shortDescription10  shortDescription100  \
0                         0                   0                    0   
1                         0                   0                    0   
2                         0                   0                    0   
3                         0                   0                    0   
4                         0                   0                    0   
...                     ...                 ...                  ...   
121108                    0                   0                    0   
121109                    0                   0                    0   
121110                    0                   0                    0   
121111                    0                   0                    0   
121112                    0                   0                    0   

        shortDescription11  shortDescription12  shortDescription13  \
0                        0                   0                   

KeyboardInterrupt: 

In [213]:
X_train_nn_2

,date,document_length,author_count,punctuation_count,headline000,headline10,headline100,headline11,headline12,headline13,...,authorsyear,authorsyoga,authorsyork,authorsyoung,authorsyouth,authorszach,authorszeba,authorszogby,authorszuburbia,authorszupkus
0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,802.0,122.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,999.0,25.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1988.0,82.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1761.0,39.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134565,632.0,189.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134567,669.0,184.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134568,5.0,199.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134569,171.0,133.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
